# Test Correspondences

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import os
from plotly.subplots import make_subplots

import planeslam.io as io
from planeslam.general import plot_3D_setup, color_legend, NED_to_ENU
from planeslam.scan import pc_to_scan
from planeslam.registration import extract_corresponding_features, get_correspondences, residual, jacobian, so3_expmap, se3_expmap
from planeslam.geometry.util import skew

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

Read in airsim LiDAR and pose data

In [ ]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

In [ ]:
# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

Extract planes 

In [ ]:
num_scans = len(PC_data)
scans = num_scans * [None]
scans_transformed = num_scans * [None]
for i in range(num_scans):
    scans[i] = pc_to_scan(NED_to_ENU(PC_data[i]))

Get correspondences

In [ ]:
# Plot 2 scans
idx_1 = 0
idx_2 = 1

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in scans[idx_1].plot_trace():
    fig.add_trace(t, row=1, col=1)

for t in scans[idx_2].plot_trace():
    fig.add_trace(t, row=1, col=2)

fig.update_layout(width=1500, height=600, scene=dict(aspectmode='data'))
fig.show()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int64)
array([ 0,  1,  2,  3,  5,  4,  8,  6, 10,  9,  7,  4], dtype=int64)

In [ ]:
n1 = scans[idx_1].planes[0].normal
n2 = scans[idx_2].planes[0].normal
c1 = scans[idx_1].planes[0].center
c2 = scans[idx_2].planes[0].center

In [ ]:
weight1 = 10
score1 = weight1 * np.dot(n1.T,n2)  # 1 to -1
score2 = np.linalg.norm(c1 - c2)  # 0 to inf

In [ ]:
n = len(scans[idx_1].planes) # source P
m = len(scans[idx_2].planes) # target Q
score_mat = np.zeros((n,m))
for i in range(n):
    for j in range(m):
        n1 = scans[idx_1].planes[i].normal
        n2 = scans[idx_2].planes[j].normal
        c1 = scans[idx_1].planes[i].center
        c2 = scans[idx_2].planes[j].center
        score_mat[i,j] = np.linalg.norm(n1 - n2) + np.linalg.norm(c1 - c2)

In [ ]:
score_mat

In [ ]:
np.argmin(score_mat, axis=0)

In [ ]:
# Extract normals and distances
# Use idx_2 as source and idx_1 as target
correspondences = get_correspondences(scans[idx_2], scans[idx_1])
correspondences = [(0,0), (1,0), (2,1), (3,2), (4,3), (5,4), (6,5)]
n_s, d_s, n_t, d_t = extract_corresponding_features(scans[idx_2], scans[idx_1], correspondences)

In [ ]:
correspondences